### This notebook is created for users to be able to use the tool easily without going through the backend code

#### Currently the run-times might be high since everything is being computed on the spot but we are working towards reducing these drastically and this will be updated soon . 
####  We have also made the back-end code available and users are free to fork the repo and make desired changes in the code for their own purposes

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display , HTML , clear_output


from Functions import shapefile_generator
from Functions import graph_plotter_cropland 
from Functions import graph_plotter_marginal
from Functions import graph_plotter_all

In [2]:
import re

# Function to sanitize the key for NetCDF variable names
def sanitize_key(key):
    # Replace any non-alphanumeric characters with underscores
    key = re.sub(r'[^a-zA-Z0-9_]', '_', key)
    return key

### Select Geography (Country and Province)

In [3]:
# Load data
geography_df = pd.read_csv('Countries&Provinces.csv')

# Get unique countries and provinces
countries = geography_df['NAME_0'].unique()
provinces = geography_df['NAME_1'].unique()

# Create dropdown widgets
country_dropdown = widgets.Dropdown(options=countries, description='Country:')
province_dropdown = widgets.Dropdown(description='Province:')
confirm_button = widgets.Button(description='Confirm Selection')

# Global variables to store selected values
selected_country = countries[0]
selected_province = provinces[0]
geography = None  # Initialize geography as None

def update_provinces(change):
    global selected_country
    selected_country = change.new
    filtered_provinces = geography_df[geography_df['NAME_0'] == selected_country]['NAME_1'].unique()

    # Add an empty option to reselect or select a new province
    province_dropdown.options = [None] + filtered_provinces.tolist()

    # Set the value to the first province in the list, or 'Select Province' if the list is empty
    province_dropdown.value = filtered_provinces[0] if filtered_provinces.any() else 'Select Province'

# Attach the update_provinces function to the country dropdown's 'value' trait
country_dropdown.observe(update_provinces, names='value')

# Function to display selected values
def on_confirm_button_click(button):
    global selected_province, geography
    clear_output(wait=True)  # Clear the output area before displaying new values

    country_value = f"<b>Selected Country:</b> {selected_country}"

    if province_dropdown.value and province_dropdown.value != 'Select Province':
        selected_province = province_dropdown.value
        province_value = f"<b>Selected Province:</b> {selected_province}"
        
        # Run the shapefile_generator code and assign the result to the global variable
        geography = shapefile_generator(selected_country, selected_province)
        
        # Additional code if needed
        
        # Display a message to indicate success
        print("Geography selected successfully!")
    else:
        selected_province = None
        province_value = "<b>Selected Province:</b> None"

    display(HTML(f"{country_value}<br>{province_value}"))

# Attach the on_confirm_button_click function to the button's 'on_click' event
confirm_button.on_click(on_confirm_button_click)

# Style dropdown options with HTML formatting
country_dropdown.style.description_width = 'initial'
country_dropdown.style.description_color = 'black'
country_dropdown.style.font_weight = 'bold'
province_dropdown.style.description_width = 'initial'
province_dropdown.style.description_color = 'black'
province_dropdown.style.font_weight = 'bold'

# Display the widgets
display(widgets.HBox([country_dropdown, province_dropdown, confirm_button]))

### Biomass Energy Potential from Cropland (Past and Future)

In [4]:
# Climate models options
climate_models = ['GFDL-ESM2M', 'HadGEM2-ES', 'NorESM1-M', 'MIROC-ESM-CHEM', 'IPSL-CM5A-LR']

# Water supply future options
water_supply_future = ['Available water content of 200 mm/m (under irrigation conditions)',
                       'Available water content of 200 mm/m (under rain-fed conditions)']

# Input level options
input_levels = ['High']

# Create separate dropdown widgets for climate models, water supply future, and input level
climate_model_dropdown = widgets.Dropdown(options=climate_models, description='Climate Model:')
water_supply_dropdown = widgets.Dropdown(options=water_supply_future, description='Water Supply Future:')
input_level_dropdown = widgets.Dropdown(options=input_levels, description='Input Level:')

# Global variables to store selected values for climate models, water supply, and input level
selected_climate_model = climate_models[0]
selected_water_supply = water_supply_future[0]
selected_input_level = input_levels[0]

# Function to update selected climate model
def update_climate_model(change):
    global selected_climate_model
    selected_climate_model = change.new

# Function to update selected water supply future
def update_water_supply(change):
    global selected_water_supply
    selected_water_supply = change.new

# Function to update selected input level
def update_input_level(change):
    global selected_input_level
    selected_input_level = change.new

# Function to display selected climate model, water supply, and input level values
def on_confirm_button_click_models(button):
    clear_output(wait=True)  # Clear the output area before displaying new values

    climate_model_value = f"<b>Selected Climate Model:</b> {selected_climate_model}"
    water_supply_value = f"<b>Selected Water Supply Future:</b> {selected_water_supply}"
    input_level_value = f"<b>Selected Input Level:</b> {selected_input_level}"

    display(HTML(f"{climate_model_value}<br>{water_supply_value}<br>{input_level_value}"))

# Create a button for climate models, water supply future, and input level confirmation
confirm_button_models = widgets.Button(description='Confirm Selection')

# Attach the update functions to the dropdowns' 'value' trait
climate_model_dropdown.observe(update_climate_model, names='value')
water_supply_dropdown.observe(update_water_supply, names='value')
input_level_dropdown.observe(update_input_level, names='value')

# Attach the on_confirm_button_click_models function to the button's 'on_click' event
confirm_button_models.on_click(on_confirm_button_click_models)

# Style dropdown options with HTML formatting for climate models, water supply future, and input level
climate_model_dropdown.style.description_width = 'initial'
climate_model_dropdown.style.description_color = 'black'
climate_model_dropdown.style.font_weight = 'bold'
water_supply_dropdown.style.description_width = 'initial'
water_supply_dropdown.style.description_color = 'black'
water_supply_dropdown.style.font_weight = 'bold'
input_level_dropdown.style.description_width = 'initial'
input_level_dropdown.style.description_color = 'black'
input_level_dropdown.style.font_weight = 'bold'

# Display the widgets for climate models, water supply future, and input level
display(widgets.HBox([climate_model_dropdown, water_supply_dropdown, input_level_dropdown, confirm_button_models]))

In [5]:
# Create a button to run the code
run_button = widgets.Button(description='Run')

# Function to run the code on button click
def on_run_button_click(button):
    clear_output(wait=True)  # Clear the output area before running the code
    global fig, xarrays
    fig, xarrays = graph_plotter_cropland(shapefile=geography, climate_model=selected_climate_model,
                                          water_supply_future=selected_water_supply, input_level=selected_input_level)
    
    # Display the figure
    display(fig)
    
    # Display the "Download NetCDF4" button after the graph is loaded
    display(download_nc_button)

# Attach the on_run_button_click function to the button's 'on_click' event
run_button.on_click(on_run_button_click)

# Create a button to download xarrays as NetCDF4
download_nc_button = widgets.Button(description='Download NetCDF4')

# Function to download xarrays as NetCDF4 on button click
def on_download_nc_button_click(button):
    if xarrays:
        # Create a list to store tuples of (key, DataArray)
        data_arrays = []

        # Iterate over the xarrays dictionary and append each tuple to the list
        for key, value in xarrays.items():
            data_arrays.append((key, value))

        try:
            # Convert each value to a DataArray before creating the Dataset
            data_arrays = [(sanitize_key(str(key)), value.to_array()) for key, value in data_arrays]

            # Create a Dataset from the list of tuples
            dataset = xr.Dataset({key: value for key, value in data_arrays})

            # Save the dataset as a NetCDF4 file
            dataset.to_netcdf('output_data.nc')
            print('NetCDF4 file downloaded successfully.')
        except Exception as e:
            print(f'Error creating dataset: {e}')
    else:
        print('No xarrays available. Please run the code first.')
        
# Attach the on_download_nc_button_click function to the button's 'on_click' event
download_nc_button.on_click(on_download_nc_button_click)

# Display the "Run" button initially
display(run_button)

Button(description='Run', style=ButtonStyle())